# "藏品集合与集合"

---



In [1]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 设置 base_url 以使输出更清晰
model.factory.base_url = 'http://test.com/museum/'




## 概述

有许多用例需要将实体分组在一起，通常是相同类别的，但有时是不同类型的。这些用例在下面的部分中举例说明，范围从拍卖批次中的对象集合，到经销商库存和博物馆藏品、展览、相关概念集合，或共享共同特征（如性别或国籍）的人群集合。

为了用一致的模式覆盖所有用例，我们引入了一个来自CIDOC-CRM外的新 `Set` 类。这避免了具有不同类型的实体集合的问题，以及对象和集合身份的语义。如果将来在核心CIDOC-CRM本体中添加等价的类，规范的新主要版本可能会改变使用它。


## 特征

集合是概念性分组，而不是物理分组。虚拟展览中的对象集合或简单的人员喜爱对象集合永远不会因为成为或不成为集合的一部分而改变其物理状态。一个人可能有一个被毁坏对象的集合，如果集合是不再存在的东西的物理聚合，那将是非常奇怪的。因此，集合是通过 `Creation` 创建的，而不是通过 `Production`，并且不能被销毁。集合在任何给定时间点可能有零个成员而不会停止存在。

像任何核心实体一样，`Set` 的实例必须有 `id` 和 `type`，可能有额外的分类，并且可以有标识符和名称。可以有关于它们的断言，最重要的是可以有成员实体。

集合的成员实体通过成员实体的 `member_of` 属性包含，该属性的值为 `Set` 的URI。理论上，成员可以是任何类，但是Linked Art API限制仅为核心实体类型。这意味着关于成员的信息发布者需要包含到 `Set` 实例的 `member_of` 属性。在两者都在同一环境中管理的情况下这不是问题，在模型中概念上也不是挑战（有反向的 `member` 属性），但是API不允许集合引用它们的成员，遵循与群体不引用它们的成员、对象不引用它们的部分相同的设计原则。这也意味着成员必须是规范中可以独立作为[记录](/api/1.0/endpoint/)的主要类之一，如 `HumanMadeObject` 或 `Activity`，而不是只在记录中使用的类，如 `Production` 或 `Name`，因为没有找到封装记录的方法。

**示例：**

展览中的对象集合。



In [2]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( of exhibition objects)
# 行为者: Art Institute of Chicago and Getty Museum (organizers)
# 对象: Set representing objects in "Manet and Modern Beauty" exhibition
# 目的: To document the grouping of objects in an exhibition
exhibition_set = model.Set(
    ident="exhset/1",
    label="Exhibition objects"
)

# 步骤 4: 创建相关对象和关系
# What identifies this set: Primary name
exhibition_set.identified_by = vocab.PrimaryName(
    content="Objects in Manet and Modern Beauty"
)

# What description: Exhibition details
exhibition_set.referred_to_by = vocab.Description(
    content="Objects in the exhibition Manet and Modern Beauty at the Art Institute of Chicago and the Getty Museum"
)

# 创作事件 for the set
creation = model.Creation()
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "2019-05-01"
timespan.end_of_the_end = "2019-05-01"
creation.timespan = timespan
exhibition_set.created_by = creation

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(exhibition_set, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/Set/exhset/1",
  "type": "Set",
  "_label": "Exhibition objects",
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Objects in Manet and Modern Beauty"
    }
  ],
  "referred_to_by": [
    {
      "type": "LinguisticObject",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435416",
          "type": "Type",
          "_label": "Description",
          "classified_as": [
            {
              "id": "http://vocab.getty.edu/aat/300418049",
              "type": "Type",
              "_label": "Brief Text"
            }
          ]
        }
      ],
      "content": "Objects in the exhibition Manet and Modern Beauty at the Art Institute of Chicago and the Getty Museum"
    }




该集合中的一个对象。



In [3]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (ing painting)
# 行为者: 爱德华·马奈（艺术家）
# 对象: Painting that is a member of the exhibition set
# 目的: To document an artwork's membership in an exhibition
painting = vocab.Painting(
    ident="spring/13",
    label="Jeanne (Spring) by Manet"
)

# 步骤 4: 创建相关对象和关系
# What identifies this painting: The name
painting.identified_by = model.Name(content="Jeanne (Spring)")

# What set this painting belongs to: Exhibition objects set
painting.member_of = model.Set(ident="exhset")

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(painting, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/HumanMadeObject/spring/13",
  "type": "HumanMadeObject",
  "_label": "Jeanne (Spring) by Manet",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300033618",
      "type": "Type",
      "_label": "Painting",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "content": "Jeanne (Spring)"
    }
  ],
  "member_of": [
    {
      "id": "http://test.com/museum/Set/exhset",
      "type": "Set"
    }
  ]
}




### 成员顺序

为了确保成员正确排序，可以将排序值作为标识符添加到成员上。这个值应该相对于集合的其他成员正确排序，字母数字最低的标识符值首先呈现，然后从那里按升序排列。这个标识符可能有一个与之关联的 `AttributeAssignment`，该分配被应应用排序值的集合 `influenced_by`。这允许同一实体同时成为多个有序集合的成员。

**示例：**

在[档案用例](/model/archives/#archival-hierarchy)中描述的Obermeyer信件应该在Stieglitz家庭信件集合中排序为"000001"。



In [4]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (rmeyer letter)
# 行为者: Bertha Obermeyer (author), Stieglitz family (correspondents)
# 对象: HumanMadeObject with sort value for archival ordering
# 目的: To document an archival letter with its position in the archive
letter = model.HumanMadeObject(
    ident="letter/2",
    label="Obermeyer 1920"
)

# 步骤 4: 创建相关对象和关系
# What identifies this letter: Primary name and sort value
letter.identified_by = vocab.PrimaryName(
    content="Obermeyer, Bertha (1920)"
)

# Sort value for ordering: "000001"
sort_value = vocab.SortValue(content="000001")

# What assigns the sort value: AttributeAssignment influenced by the set
assignment = model.AttributeAssignment()
assignment.influenced_by = model.Set(
    ident="archive_sfl",
    label="Stieglitz Family Letters"
)
sort_value.assigned_by = assignment
letter.identified_by = sort_value

# What set this letter belongs to: Stieglitz Family Letters archive
letter.member_of = model.Set(
    ident="archive_sfl",
    label="Stieglitz Family Letters"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(letter, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/HumanMadeObject/letter/2",
  "type": "HumanMadeObject",
  "_label": "Obermeyer 1920",
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Obermeyer, Bertha (1920)"
    },
    {
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300456575",
          "type": "Type",
          "_label": "Sort Value"
        }
      ],
      "content": "000001",
      "assigned_by": [
        {
          "type": "AttributeAssignment",
          "influenced_by": [
            {
              "id": "http://test.com/museum/Set/archive_sfl",
              "type": "Set",
              "_label": "Stieglitz Family Letters"
            }
          ]
        }
      ]
    }
  ],
  "memb



### 原型成员

关于集合中任何特定成员的信息可能不可用，但是可能知道关于集合中实体的通用信息。例如，特定集合中的对象可能由同一个人创作，被分类为相同类型，或者有相同的所有者。作品可能用相同语言编写，关于相同主题等等。由于任何实体都可以成为集合的成员，这为描述被分组在一起的事物类型提供了很大的自由度。这对于档案经常是真实的，但对于使集合的基本原理更明显也很有价值，例如绘画部门策展的对象（通常）是绘画。

原型成员的描述嵌入在集合中作为 `members_exemplified_by` 属性的值。

这不是集合的"重点"成员，如对象集合中的著名作品。为此，使用[相关对象](/model/assertion/)模式。使用这种方法并不意味着集合的*每个*成员都具有原型成员的所有特征，但是应该注意不要包含通常不真实的信息。

**示例：**

展览的对象通常是（但不排除）马奈的绘画。



In [5]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( of exhibition objects)
# 行为者: 爱德华·马奈 (typical artist in the set)
# 对象: Set with prototypical member description
# 目的: To describe what kinds of objects are typically in this set
exhibition_set = model.Set(
    ident="exhset/2",
    label="Exhibition objects"
)

# 步骤 4: 创建相关对象和关系
# 什么 prototypical member: A painting by Manet
prototype_painting = vocab.Painting()

# Production event for prototype: Manet created it
production = model.Production()
production.carried_out_by = model.Person(
    ident="http://vocab.getty.edu/ulan/500010363",
    label="Manet"
)
prototype_painting.produced_by = production

# Set members are exemplified by this prototype
exhibition_set.members_exemplified_by = prototype_painting

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(exhibition_set, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/Set/exhset/2",
  "type": "Set",
  "_label": "Exhibition objects",
  "members_exemplified_by": [
    {
      "type": "HumanMadeObject",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300033618",
          "type": "Type",
          "_label": "Painting",
          "classified_as": [
            {
              "id": "http://vocab.getty.edu/aat/300435443",
              "type": "Type",
              "_label": "Type of Work"
            }
          ]
        }
      ],
      "produced_by": {
        "type": "Production",
        "carried_out_by": [
          {
            "id": "http://vocab.getty.edu/ulan/500010363",
            "type": "Person",
            "_label": "Manet"
          }
        ]
      }
    }
  ]
}




## 对象集合

集合可以用来描述构成策展藏品的对象集合。这不一定与机构拥有的对象集合相同，因为可能有由其他组织或个人拥有但由机构照看的对象，也不是机构目前拥有保管权的对象集合，因为借给其他组织展览的对象仍然是概念性对象集合的一部分。对象与机构之间的关系细节记录在对象上，而集合为藏品本身提供身份，独立于成员对象。对象可以同时成为多个集合的一部分——私人所有者的个人收藏和博物馆的公共收藏。因此虽然大多数对象都是由机构拥有并由其保管，但这不是确定的，不应该被推断。

机构经常被分成部门，每个部门将管理整体藏品的一部分。这些藏品部分作为单独的集合管理，而不是以某种方式在单个集合内。部门藏品可能是机构藏品的成员，就像部门是组织的 `member_of` 一样。能够描述对象在每个上下文中的属性是很有用的，并允许从组织图表中分离的库存管理结构。部门也可能构思他们对象的进一步集合，没有任何直接对应关系，很可能同一个对象同时成为多个集合的一部分。

**示例：**

国家博物馆的完整藏品。



In [6]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (ksmuseum Collection)
# 行为者: Rijksmuseum (managing institution)
# 对象: CollectionSet representing the full museum collection
# 目的: To document the institutional collection as a set
collection = vocab.CollectionSet(
    ident="rijks_objects/1",
    label="Collection of the Rijksmuseum"
)

# 步骤 4: 创建相关对象和关系
# What identifies this collection: Primary name
collection.identified_by = vocab.PrimaryName(
    content="Collection of the Rijksmuseum"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(collection, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/Set/rijks_objects/1",
  "type": "Set",
  "_label": "Collection of the Rijksmuseum",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300025976",
      "type": "Type",
      "_label": "Collection"
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Collection of the Rijksmuseum"
    }
  ]
}




国家博物馆的绘画，由绘画部门策展。



In [7]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (ksmuseum Paintings Collection)
# 行为者: Rijksmuseum Paintings Department (curator)
# 对象: CollectionSet representing paintings managed by Paintings department
# 目的: To document a departmental collection subset
paintings_collection = vocab.CollectionSet(
    ident="rijks_paintings/1",
    label="Paintings of the Rijksmuseum"
)

# 步骤 4: 创建相关对象和关系
# What identifies this collection: Primary name
paintings_collection.identified_by = vocab.PrimaryName(
    content="Paintings of the Rijksmuseum"
)

# What larger collection this is part of: Full Rijksmuseum collection
paintings_collection.member_of = model.Set(
    ident="rijks_objects",
    label="Collection of the Rijksmuseum"
)

# Who curates this collection: Paintings Department
curating = vocab.Curating()
curating.carried_out_by = model.Group(
    ident="rijks_paintings_dept",
    label="Paintings Department"
)
paintings_collection.used_for = curating

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(paintings_collection, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/Set/rijks_paintings/1",
  "type": "Set",
  "_label": "Paintings of the Rijksmuseum",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300025976",
      "type": "Type",
      "_label": "Collection"
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Paintings of the Rijksmuseum"
    }
  ],
  "member_of": [
    {
      "id": "http://test.com/museum/Set/rijks_objects",
      "type": "Set",
      "_label": "Collection of the Rijksmuseum"
    }
  ],
  "used_for": [
    {
      "type": "Activity",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300054277",
          "type": "Type",
          "_label": "Curating"
        }
      ],
      "carried_out_by": [
   



夜巡是绘画集合的成员。



In [8]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( Night Watch)
# 行为者: 伦勃朗（艺术家）, Rijksmuseum Paintings Department (curator)
# 对象: Painting that is a member of the paintings collection
# 目的: To document an artwork's membership in a curated collection
painting = vocab.Painting(
    ident="nightwatch/16",
    label="Night Watch by Rembrandt"
)

# 步骤 4: 创建相关对象和关系
# What identifies this painting: Primary name
painting.identified_by = vocab.PrimaryName(
    content="The Night Watch"
)

# What collection this painting belongs to: Rijksmuseum Paintings
painting.member_of = model.Set(
    ident="rijks_paintings",
    label="Paintings of the Rijksmuseum"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(painting, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/HumanMadeObject/nightwatch/16",
  "type": "HumanMadeObject",
  "_label": "Night Watch by Rembrandt",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300033618",
      "type": "Type",
      "_label": "Painting",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "The Night Watch"
    }
  ],
  "member_of": [
    {
      "id": "http://test.com/museum/Set/rijks_paintings",
      "type": "Set",
      "_label": "Paintings of the Rijksmuseum"
    }
  ]
}





## 其他用例

### 人群集合

群体是其他 `Group` 和 `Person` 的集合，可以采取行动。这意味着集合的特征也可用于群体，如 `members_exemplified_by`。这允许我们具体说明大型群体的原型成员，如国籍或"未识别"人员。

/// warning
Set是Group的新超类仅在使用Linked Art本体时成立，在CIDOC-CRM基础本体中不成立，因为Linked Art引入了Set的概念。
///


**示例：**

"14世纪意大利人"群体的原型成员是在1300年至1400年间在意大利出生的人。



In [9]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (h Century Italians group)
# 行为者: Unidentified Italian people from 14th century
# 对象: Group with prototypical member description
# 目的: To describe the characteristics of typical group members
italian_group = model.Group(
    ident="14thc_italian/1",
    label="14th c italians"
)

# 步骤 4: 创建相关对象和关系
# What identifies this group: Primary name
italian_group.identified_by = vocab.PrimaryName(
    content="Unidentified 14th Century Italian"
)

# 什么 prototypical member: A person born 1300-1399 in Italy
prototype_person = model.Person()

# Birth event for prototype: Born between 1300-1399 in Italy
birth = model.Birth()
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1300-01-01T00:00:00Z"
timespan.end_of_the_end = "1399-12-31T23:59:59Z"
birth.timespan = timespan

# Where born: Italy
birth.took_place_at = model.Place(
    ident="https://vocab.getty.edu/tgn/1000080",
    label="Italy"
)
prototype_person.born = birth

# Group members exemplified by this prototype
italian_group.members_exemplified_by = prototype_person

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(italian_group, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://test.com/museum/Group/14thc_italian/1",
  "type": "Group",
  "_label": "14th c italians",
  "identified_by": [
    {
      "type": "Name",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300404670",
          "type": "Type",
          "_label": "Primary Name"
        }
      ],
      "content": "Unidentified 14th Century Italian"
    }
  ],
  "members_exemplified_by": [
    {
      "type": "Person",
      "born": {
        "type": "Birth",
        "timespan": {
          "type": "TimeSpan",
          "begin_of_the_begin": "1300-01-01T00:00:00Z",
          "end_of_the_end": "1399-12-31T23:59:59Z"
        },
        "took_place_at": [
          {
            "id": "https://vocab.getty.edu/tgn/1000080",
            "type": "Place",
            "_label": "Italy"
          }
        ]
      }
    }
  ]
}




### 档案

Set类广泛用于建模[档案](/model/archives/)。

### 拍卖批次

[拍卖批次](/model/provenance/auctions.html#set-of-objects)中的对象集合也被建模为集合。这些不像博物馆藏品那样策展，不一定曾经物理聚集在一起，甚至可能不是物理的而是数字对象，但作为单一实体被拍卖。同样，[展览](/model/exhibition/#objects)中使用的对象集合也被建模为集合。

### 藏品特定信息

关于实体特定于它们所属集合上下文的信息，如对象对于该特定藏品的入藏号，可以使用关于[断言](/model/assertion/#context-specific-assertions)页面中描述的 `AttributeAssignment` 模式来描述，方式与上面描述的集合内排序值相同。